---
title: Law Tracker
format: html
---




Coming soon.


In [ ]:
import warnings
warnings.filterwarnings("ignore", message="Workbook contains no default style.*")
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

# Data loading and processing
path = "../data"

dataframes = []
for file in os.listdir(path):
    if file.endswith(".xlsx"):
        filepath = os.path.join(path, file)
        df = pd.read_excel(filepath)
        dataframes.append(df)

df = pd.concat(dataframes, ignore_index=True)
df.to_excel("df.xlsx", index=False)
df = df.rename(columns={"Unnamed: 6": "Auction Price €/tCO2"})
df = df.rename(columns={"Unnamed: 1": "Date"})

# Prepare weekly data
df1 = df[['Date', 'Auction Price €/tCO2']].dropna()
df1 = df1.iloc[2:]
df1['Date'] = pd.to_datetime(df1['Date'], errors='coerce')
df1 = df1.dropna(subset=['Date'])
df1['year'] = df1['Date'].dt.isocalendar().year
df1['week'] = df1['Date'].dt.isocalendar().week
weeklyaverage = df1.groupby(['year', 'week'])['Auction Price €/tCO2'].mean().reset_index()
df1 = pd.merge(df1, weeklyaverage, on=['year', 'week'], how='left', suffixes=('', '_weekly_avg'))
df1.rename(columns={'Auction Price €/tCO2_weekly_avg': 'Weekly Avg Auction Price €/tCO2'}, inplace=True)
df1.sort_values(by=['year', 'week'], inplace=True)
df1['yearweek'] = df1['year'].astype(str) + '-W' + df1['week'].astype(str).str.zfill(2)
df1['yearweek_date'] = pd.to_datetime(df1['year'].astype(str) + df1['week'].astype(str).str.zfill(2) + '1', format='%G%V%u')

# Prepare quarterly data
df2 = df[['Date', 'Auction Price €/tCO2']].copy()
df2['Date'] = pd.to_datetime(df2['Date'], errors='coerce')
df2.dropna(subset=['Date'], inplace=True)
df2['year'] = df2['Date'].dt.isocalendar().year
df2['quarter'] = df2['Date'].dt.quarter
quarteraverage = df2.groupby(['year', 'quarter'])['Auction Price €/tCO2'].mean().reset_index()
df2 = pd.merge(df2, quarteraverage, on=['year', 'quarter'], how='left', suffixes=('', '_quarteravg'))
# The column after merge is 'Auction Price €/tCO2_quarteravg'
# So, just check columns:
print("Columns in df2:", df2.columns)
# Proceed with plotting

# Create plotly figure
fig = go.Figure()

# Add Weekly Average trace
fig.add_trace(go.Scatter(
    x=df1['yearweek_date'],
    y=df1['Weekly Avg Auction Price €/tCO2'],
    mode='lines+markers',
    name='Weekly Average',
    line=dict(color='lightblue'),
    hovertemplate='Week: %{x|%Y-%m-%d}<br>Price: €%{y:.2f}<extra></extra>'
))

# Add Quarterly Average trace
# Use the correct column name from the merge
quarterly_column_name = 'Auction Price €/tCO2_quarteravg'
print("Using column:", quarterly_column_name)
fig.add_trace(go.Scatter(
    x=df2['quarter_start'],
    y=df2[quarterly_column_name],
    mode='lines+markers',
    name='Quarterly Average',
    line=dict(shape='hv', color='darkblue'),
    hovertemplate='Quarter Start: %{x|%Y-%m-%d}<br>Price: €%{y:.2f}<extra></extra>'
))

# Generate HTML for the plot
plot_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')

# Build the full HTML page
full_html = f"""
<html>
<head>
  <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
<h2>Comparing EU ETS Allowance Closing Prices Based on Weekly v. Quarterly Averages</h2>

<!-- Plot container -->
<div id="plot">{plot_html}</div>

<!-- Hidden inputs to store selected prices from plot -->
<input type="hidden" id="selectedWeeklyPrice" value="">
<input type="hidden" id="selectedQuarterlyPrice" value="">

<!-- Emission input fields -->
<div>
  <label>Weight of Emissions (tonnes): <input type="number" id="weight" value="50"></label><br>
  <label>Emission Factor (tCO₂e/tonne): <input type="number" id="ef" value="2.3" step="0.1"></label><br>
  <button onclick="calculateCBAM()">Calculate</button>
</div>

<!-- Results -->
<p id="result_weekly"></p>
<p id="result_quarterly"></p>

<!-- Scripts for interaction -->
<script>
  // Add click event listener to the plot
  document.querySelector('#plot').on('plotly_click', function(data){
    const point = data.points[0];
    if(point.data.name === 'Weekly Average'){
        const weeklyPrice = point.y;
        document.getElementById('selectedWeeklyPrice').value = weeklyPrice;
        alert('Selected Weekly Price: €' + weeklyPrice.toFixed(2));
    } else if(point.data.name === 'Quarterly Average'){
        const quarterlyPrice = point.y;
        document.getElementById('selectedQuarterlyPrice').value = quarterlyPrice;
        alert('Selected Quarterly Price: €' + quarterlyPrice.toFixed(2));
    }
  });

  function calculateCBAM() {
    const weight = parseFloat(document.getElementById('weight').value);
    const ef = parseFloat(document.getElementById('ef').value);
    const weeklyPrice = parseFloat(document.getElementById('selectedWeeklyPrice').value);
    const quarterlyPrice = parseFloat(document.getElementById('selectedQuarterlyPrice').value);

    if (isNaN(weeklyPrice) || isNaN(quarterlyPrice)) {
        alert('Please click on the plot points to select both weekly and quarterly prices.');
        return;
    }

    const totalWeekly = weight * ef * weeklyPrice;
    const totalQuarterly = weight * ef * quarterlyPrice;

    document.getElementById('result_weekly').innerText =
      `Carbon Certificate Price based on Weekly EU ETS Allowance Closing Price Averages: €${totalWeekly.toFixed(2)}`;

    document.getElementById('result_quarterly').innerText =
      `Carbon Certificate Price based on Quarterly EU ETS Allowance Closing Price Averages: €${totalQuarterly.toFixed(2)}`;
  }
</script>
</body>
</html>


In [ ]:
# Save the HTML file
with open("carbon_price_comparison.html", "w") as f:
    f.write(full_html)

print("Open 'carbon_price_comparison.html' in your browser to interact.")